In [1]:
from torchvision import transforms, models
# from sklearn.manifold import TSNE
from PIL import Image
from torch import nn
import pandas as pd
import numpy as np
import argparse
import random
import torch
import time
import copy
import os

from dataloader import *
from utils import *

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
activation = {}
def getActivation(name):
    # the hook signature
    def hook(model, input, output):
#         print(input)
        activation[name] = input[0].detach()
    return hook

In [3]:
model = models.resnet18(pretrained = True)
model.fc.register_forward_hook(getActivation('fc'))
model = model.to("cuda")

In [4]:
ecup = f"../../imagery/{'NG'}/"
image_names = [ecup + _ for _ in os.listdir(ecup)]
data = Dataloader(image_names, "../../clean_data/wealth_random_points_cumsum_jenks.csv", "test", batch_size = 32)

In [5]:
all_stds, all_means = [], []
all_stats = []

for batch in range(data.get_num("train") - 1):
# for batch in range(50):

    try:

        feats = model(data.load_data(batch, "train")[0].to("cuda"))
        batch_fc = activation["fc"].detach().cpu().numpy()
        stds = np.std(batch_fc, 0)
        means = np.mean(batch_fc, 0)
        all_stds.append(stds)
        all_means.append(means)
        all_stats.append(np.concatenate([stds, means]))
        
    except:
            
        pass

# country_feats[country].append(activation["fc"][0].detach().cpu().numpy())

In [6]:
from sklearn.manifold import TSNE

In [7]:
all_stats = np.array(all_stats)

In [8]:
pd.DataFrame(all_stats).to_csv("./NG_stats.csv", index = False)

In [9]:
# all_stats.shape

In [10]:
# tsne = TSNE(n_components = 2)
# values = all_stats
# print(values.shape)
# tsne_vals = tsne.fit_transform(np.array(values))

In [11]:
# pd.DataFrame(tsne_vals).to_csv("./tfeats.csv", index = False)

In [12]:
# feats = pd.DataFrame(tsne_vals)
# feats = feats.rename(columns = {0: "tsne0", 1: "tsne1"})
# feats.head()

In [13]:
# import matplotlib.pyplot as plt

In [14]:
# plt.scatter(x = feats["tsne0"], y = feats["tsne1"], s = 2)